# Azure Machine Learning Pipeline with AutoMLStep
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook we will do:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.40.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-190579
aml-quickstarts-190579
southcentralus
cdbe0b43-92a0-4715-838a-f2648cc7ad21


## Create an Azure ML experiment
Let's create an experiment and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

In [3]:
# Choose a name for the run history container in the workspace.
experiment_name = 'automl-pipeline-classification'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl-pipeline-classification,quick-starts-ws-190579,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
Create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for AutoML run.

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Choose a name for CPU cluster
amlcompute_cluster_name = "automl-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file

found = False
key = "BankMarketing Dataset"
description_text = "Bank Marketing DataSet"

if key in ws.datasets.keys(): 
        found = True
        print('Found existing dataset, use it.')
        dataset = ws.datasets[key] 

if not found:
        print('Not Found dataset. Create New One !!!')
        # Create AML Dataset and register it into Workspace
        Bankmarketing_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(Bankmarketing_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

Found existing dataset, use it.


,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [6]:
dataset.take(5).to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


## Train
This creates a general AutoML settings object.
These inputs must match what was used when training in the portal. `label_column_name` has to be `y` for example.

In [7]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

Define outputs for the AutoMLStep using TrainingOutput.

In [8]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [9]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [10]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [11]:
pipeline_run = experiment.submit(pipeline, show_output=True)

Created step automl_module [9bb26ffe][e42bf1f0-c04d-4bed-9095-42bea8c78ad3], (This step will run and generate new outputs)
Submitted PipelineRun 255a97c9-71b2-4946-8b2f-897fb6c67d15
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/255a97c9-71b2-4946-8b2f-897fb6c67d15?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-190579/workspaces/quick-starts-ws-190579&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


In [12]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

KeyError: 'log_files'

In [13]:
pipeline_run.wait_for_completion(show_output=True)

PipelineRunId: 255a97c9-71b2-4946-8b2f-897fb6c67d15
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/255a97c9-71b2-4946-8b2f-897fb6c67d15?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-190579/workspaces/quick-starts-ws-190579&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: Running


StepRunId: f004abad-ac1e-4b47-9907-f51cee547605
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/f004abad-ac1e-4b47-9907-f51cee547605?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-190579/workspaces/quick-starts-ws-190579&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished

Warnings:
No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/py

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. Examine the outputs by retrieve output data and running some tests.

In [14]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/f004abad-ac1e-4b47-9907-f51cee547605/metrics_data, 1 files out of an estimated total of 1


In [15]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df.head(5)

,f004abad-ac1e-4b47-9907-f51cee547605_0,f004abad-ac1e-4b47-9907-f51cee547605_2,f004abad-ac1e-4b47-9907-f51cee547605_3,f004abad-ac1e-4b47-9907-f51cee547605_5,f004abad-ac1e-4b47-9907-f51cee547605_6,f004abad-ac1e-4b47-9907-f51cee547605_4,f004abad-ac1e-4b47-9907-f51cee547605_7,f004abad-ac1e-4b47-9907-f51cee547605_10,f004abad-ac1e-4b47-9907-f51cee547605_8,f004abad-ac1e-4b47-9907-f51cee547605_11,...,f004abad-ac1e-4b47-9907-f51cee547605_40,f004abad-ac1e-4b47-9907-f51cee547605_47,f004abad-ac1e-4b47-9907-f51cee547605_46,f004abad-ac1e-4b47-9907-f51cee547605_1,f004abad-ac1e-4b47-9907-f51cee547605_18,f004abad-ac1e-4b47-9907-f51cee547605_23,f004abad-ac1e-4b47-9907-f51cee547605_22,f004abad-ac1e-4b47-9907-f51cee547605_31,f004abad-ac1e-4b47-9907-f51cee547605_26,f004abad-ac1e-4b47-9907-f51cee547605_41
precision_score_macro,[0.7781441711329087],[0.771526544069397],[0.4440060698027314],[0.7856451172940535],[0.6643997821350762],[0.8047197211733953],[0.8065451980757572],[0.7982640315624551],[0.7845613466137928],[0.6182658387401366],...,[0.4440060698027314],[0.7932600501884566],[0.8045780002534533],[0.7826005666081467],[0.7881835838009958],[0.6517407903148638],[0.7806355042016807],[0.7807987724455402],[0.782781125204717],[0.7815893103850868]
f1_score_micro,[0.9101669195751139],[0.8977238239757208],[0.8880121396054628],[0.908649468892261],[0.7915022761760242],[0.9071320182094081],[0.9119878603945372],[0.9083459787556905],[0.9104704097116844],[0.7599393019726859],...,[0.8880121396054628],[0.9122913505311078],[0.9153262518968134],[0.9116843702579667],[0.9132018209408195],[0.7602427921092565],[0.9110773899848255],[0.9107738998482551],[0.9101669195751139],[0.909256449165402]
weighted_accuracy,[0.9512815952194833],[0.9730611889183236],[0.9843450583187134],[0.9619470207827714],[0.7771157844366237],[0.9720244705544736],[0.9679994692811393],[0.9686663170697974],[0.9574977663473424],[0.7635899267812627],...,[0.9843450583187134],[0.9606921972025276],[0.9631743656351719],[0.952375685487909],[0.9546457273395061],[0.7398193325382515],[0.9517028590639043],[0.9528363853308665],[0.9565733773437545],[0.9576220529782975]
precision_score_weighted,[0.9051980543721705],[0.877014103638037],[0.788565560086672],[0.8973758906640772],[0.9153658649634193],[0.8930395334907932],[0.9005211086889047],[0.8950256468849379],[0.9021040535657128],[0.8809627535230434],...,[0.788565560086672],[0.9032575330132339],[0.9066260288848306],[0.9067326786550023],[0.907597716175493],[0.9152162115762714],[0.9062625859144872],[0.9051333780455183],[0.9021382069947883],[0.9002533866705345]
AUC_micro,[0.9795361989126856],[0.9630975336245426],[0.9630417172291673],[0.9779290367296751],[0.902177530216611],[0.9732255383035407],[0.9758368429657296],[0.9741933909150988],[0.9790036405000448],[0.8370880604953935],...,[0.972845047331106],[0.9801307448403223],[0.9805304860217233],[0.9783641467160662],[0.978789217119791],[0.8721719808142654],[0.9789469951483026],[0.9754714574204258],[0.9780501564655144],[0.9781238414759107]


### Retrieve the Best Model

In [16]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/f004abad-ac1e-4b47-9907-f51cee547605/model_data, 1 files out of an estimated total of 1


In [17]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='classification', working_dir='/m...
                                                  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('27', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(boosting_type='goss', colsample_bytree=0.8911111111111111, learning_rate=0.03158578947368421, max_bin=280, max_depth=9, min_child_weight=9, min_data_in_leaf=1e-05, min_split_gain=0.2631578947368421, n_estimators=100, n_jobs=1, num_leaves=89, problem_info=ProblemInfo(gpu_training_param_dict={'

In [18]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, task='classification')),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('27', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(boosting_type='goss', colsample_bytree=0.8911111111111111, learning_rate=0.03158578947368421, max_bin=280, max_depth=9, min_child_weight=9, min_data_in_leaf=1e-05, min_split_gain=0.2631578947368421, n_estimators=100, n_jobs=1, num_leaves=89, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None, reg_alpha=0.10526315789473684, reg_lambda=0.21052631578947367, subsample=1))], verbose=False)), ('30', Pipeline(memory=None, steps=[('standardsca

### Test the Model
#### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [20]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file

found_test = False
key_test = "bankmarketing test"
description_test = "bankmarketing test"

if key_test in ws.datasets.keys():
        found_test = True
        print('Found existing dataset, use it.')
        dataset_test = ws.datasets[key_test]

if not found_test:
        print('Not Found dataset. Create New One !!!')
        # Create AML Dataset and register it into Workspace
        Bankmarketing_test_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_test.csv'
        dataset_test = Dataset.Tabular.from_delimited_files(Bankmarketing_test_data)        
        #Register Dataset in Workspace
        dataset_test = dataset_test.register(workspace=ws, name=key_test, description=description_test)

df_test = dataset_test.to_pandas_dataframe()
df_test.head(5)

Not Found dataset. Create New One !!!


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,51,retired,married,basic.4y,no,no,no,cellular,jul,wed,...,1,13,1,success,-2.90,92.47,-33.60,1.08,5076.20,no
1,53,self-employed,divorced,university.degree,no,no,no,cellular,jul,mon,...,3,999,0,nonexistent,1.40,93.92,-42.70,4.96,5228.10,no
2,32,services,married,basic.9y,no,no,no,cellular,jul,tue,...,1,999,0,nonexistent,1.40,93.92,-42.70,4.96,5228.10,no
3,44,management,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.10,93.20,-42.00,4.19,5195.80,no
4,36,admin.,single,university.degree,no,no,no,cellular,may,fri,...,3,999,1,failure,-1.80,92.89,-46.20,1.31,5099.10,no


In [21]:
df_test = df_test[pd.notnull(df_test['y'])]
df_test.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,51,retired,married,basic.4y,no,no,no,cellular,jul,wed,...,1,13,1,success,-2.90,92.47,-33.60,1.08,5076.20,no
1,53,self-employed,divorced,university.degree,no,no,no,cellular,jul,mon,...,3,999,0,nonexistent,1.40,93.92,-42.70,4.96,5228.10,no
2,32,services,married,basic.9y,no,no,no,cellular,jul,tue,...,1,999,0,nonexistent,1.40,93.92,-42.70,4.96,5228.10,no
3,44,management,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.10,93.20,-42.00,4.19,5195.80,no
4,36,admin.,single,university.degree,no,no,no,cellular,may,fri,...,3,999,1,failure,-1.80,92.89,-46.20,1.31,5099.10,no


In [22]:
y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

#### Testing Our Best Fitted Model

We will use confusion matrix to see how our model works.

In [23]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

In [24]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

,0,1
0,3542,94
1,244,240


## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [25]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bankmarketing Train,5f2feffb-8e8c-48f2-9140-4f8d8ab0c8c1,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [26]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [27]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [28]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  a3a277be-e73c-4a09-8094-777259d497ea


Use the run id to monitor the status of the new run.

In [29]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …